In [1]:
%pip install -qU langchain-openai langchain langgraph langchain_community langgraph-supervisor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 976.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 9.8 MB/s eta 0:00:00


In [2]:
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from google.colab import userdata
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor
from langchain_core.tools import tool
from langchain_core.messages import (
    HumanMessage,
)
from langgraph.prebuilt import create_react_agent
import requests
key = userdata.get('OPENAI_API_KEY')

In [3]:
model = ChatOpenAI(api_key=key,model="gpt-4o-mini")

# Tool

In [ ]:
# from langchain_core.tools import tool
# import requests
# import os

# tool
# def search_news_by_keyword(keyword: str, region: str = "en-US") -> str:
#     """
#     Search Google News for articles matching a keyword using the google-news13 API.

#     Args:
#         keyword (str): The keyword to search for.
#         region (str): Language-region code (default: 'en-US').

#     Returns:
#         str: A summary of the top 3 news articles with title, summary, and link.
#     """
#     url = "https://google-news13.p.rapidapi.com/search"
#     querystring = {"keyword": keyword, "lr": region}
#     headers = {
#         "x-rapidapi-key": "api key",
#         "x-rapidapi-host": "google-news13.p.rapidapi.com"
#     }

#     try:
#         response = requests.get(url, headers=headers, params=querystring)
#         response.raise_for_status()
#         data = response.json()
#         items = data.get("items", [])

#         if not items:
#             return f"No news found for keyword: {keyword}"

#         result = f"📰 Top News for '{keyword}':\n"
#         for idx, item in enumerate(items[:3], start=1):  # Top 3 items
#             title = item.get("title", "No title")
#             snippet = item.get("snippet", "No summary")
#             link = item.get("newsUrl", "No link")
#             result += f"\n{idx}. {title}\n📄 {snippet}\n🔗 {link}\n"
#         return result.strip()

#     except requests.exceptions.RequestException as e:
#         return f"❌ Error fetching news: {str(e)}"
#     except (KeyError, TypeError) as e:
#         return f"❌ Unexpected response format: {str(e)}"
from langchain_core.tools import tool
import requests
import time

@tool
def search_news_by_keyword(keyword: str, region: str = "en-US") -> dict:
    """
    Search Google News for articles matching a keyword using the google-news13 API.

    Args:
        keyword (str): The keyword to search for.
        region (str): Language-region code (default: 'en-US').

    Returns:
        dict: A JSON object containing the keyword and a list of top 3 news articles.
    """
    url = "https://google-news13.p.rapidapi.com/search"
    querystring = {"keyword": keyword, "lr": region}
    headers = {
        "x-rapidapi-key": "api key",  # Replace with your RapidAPI key
        "x-rapidapi-host": "google-news13.p.rapidapi.com"
    }

    try:
        response = requests.get(url, headers=headers, params=querystring)
        response.raise_for_status()
        data = response.json()
        items = data.get("items", [])

        articles = []
        for item in items[:3]:  # Limit to top 3 articles
            timestamp = item.get("timestamp")
            readable_time = time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(int(timestamp) / 1000)) if timestamp else None

            articles.append({
                "title": item.get("title", "No title"),
                "snippet": item.get("snippet", "No summary"),
                "news_url": item.get("newsUrl", "No link"),
                "thumbnail": item.get("images", {}).get("thumbnail", ""),
                "publisher": item.get("publisher", "Unknown"),
                "timestamp": timestamp,
                "published_time": readable_time
            })

        return {
            "query": keyword,
            "total": len(articles),
            "articles": articles
        }

    except requests.exceptions.RequestException as e:
        return {"error": f"Request failed: {str(e)}"}
    except (KeyError, TypeError) as e:
        return {"error": f"Unexpected response format: {str(e)}"}


# Agent

In [5]:
# google_assistant = create_react_agent(
#     model=model,
#     tools=[search_news_by_keyword],
#     prompt="You are a helpful Google search assistant. Your job is to help users find Google articles based on their search queries.",
#     name="google_assistant"
# )
google_assistant = create_react_agent(
    model=model,
    tools=[search_news_by_keyword],
    prompt=(
        "You are a helpful Google News assistant. Your task is to help users find recent news articles based on their search queries.\n\n"
        "Use the `search_news_by_keyword` tool by passing the keyword provided by the user.\n\n"
        "Always return the results strictly in this JSON format:\n"
        "{\n"
        '  "query": string,\n'
        '  "total": integer,\n'
        '  "articles": [\n'
        '    {\n'
        '      "title": string,\n'
        '      "snippet": string,\n'
        '      "news_url": string,\n'
        '      "thumbnail": string,\n'
        '      "publisher": string,\n'
        '      "timestamp": integer,\n'
        '      "published_time": string\n'
        '    },\n'
        "    ... (up to 3 items)\n"
        "  ]\n"
        "}\n\n"
        "Do not include markdown, extra explanations, or formatting — only return clean JSON."
    ),
    name="google_assistant"
)


# Agent Invoke

In [6]:
response = google_assistant.invoke(
    {
        "messages": [
            HumanMessage(content="mental health")
            ]
    }
    )
response["messages"]

[HumanMessage(content='mental health', additional_kwargs={}, response_metadata={}, id='27ed6741-c76b-45b7-adcc-4465e63aa07c'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_pk3te8VeRpuyuKU7rnlhlkzR', 'function': {'arguments': '{"keyword":"mental health"}', 'name': 'search_news_by_keyword'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 261, 'total_tokens': 278, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BjeMUte1jcjAWefcUu5C6We9Py0Ej', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='google_assistant', id='run--411b8227-9f1a-4ff7-9e12-0478d1ea0bf9-0', tool_calls=[{'name': 'search_news_by_keyword', 'args': {

In [7]:
response["messages"][-1].content

'{\n  "query": "mental health",\n  "total": 3,\n  "articles": [\n    {\n      "title": "A father’s poor mental health may harm his children’s development, study finds",\n      "snippet": "A father\'s poor mental health may harm child development, a new study has found. Here\'s how fathers, too, can get the support they need.",\n      "news_url": "https://www.cnn.com/2025/06/16/health/father-mental-health-child-development-wellness",\n      "thumbnail": "https://news.google.com/api/attachments/CC8iI0NnNDNXblpwUVdORlRURjFaMXBtVFJERUF4aW5CU2dLTWdB=-w280-h168-p-df-rw",\n      "publisher": "CNN",\n      "timestamp": 1750086000000,\n      "published_time": "2025-06-16 15:00:00"\n    },\n    {\n      "title": "Pain management startup Sword Health expands into mental health, raises $40 million",\n      "snippet": "Sword Health is pulling in additional capital as it expands into mental health care with a new artificial intelligence tool.",\n      "news_url": "https://www.cnbc.com/2025/06/17/swo

# Class Local

# Google News Tool. PY

In [ ]:
from langchain_core.tools import tool
import requests
import os
from config.setting import RAPIDAPI_KEY
@tool
def search_news_by_keyword(keyword: str, region: str = "en-US") -> str:
    """
    Search Google News for articles matching a keyword using the google-news13 API.

    Args:
        keyword (str): The keyword to search for.
        region (str): Language-region code (default: 'en-US').

    Returns:
        str: A summary of the top 3 news articles with title, summary, and link.
    """
    rapidapi_key = RAPIDAPI_KEY
    url = "https://google-news13.p.rapidapi.com/search"
    querystring = {"keyword": keyword, "lr": region}
    headers = {
        "x-rapidapi-key": rapidapi_key,
        "x-rapidapi-host": "google-news13.p.rapidapi.com"
    }

    try:
        response = requests.get(url, headers=headers, params=querystring)
        response.raise_for_status()
        data = response.json()
        items = data.get("items", [])

        if not items:
            return f"No news found for keyword: {keyword}"

        result = f"📰 Top News for '{keyword}':\n"
        for idx, item in enumerate(items[:3], start=1):  # Top 3 items
            title = item.get("title", "No title")
            snippet = item.get("snippet", "No summary")
            link = item.get("newsUrl", "No link")
            result += f"\n{idx}. {title}\n📄 {snippet}\n🔗 {link}\n"
        return result.strip()

    except requests.exceptions.RequestException as e:
        return f"❌ Error fetching news: {str(e)}"
    except (KeyError, TypeError) as e:
        return f"❌ Unexpected response format: {str(e)}"


# Run google news agent.py

In [ ]:
from dotenv import load_dotenv
load_dotenv()

from agents.google_news_agent import GoogleNewsAgent
def main():
    agent = GoogleNewsAgent()
    query = "mental health"
    result = agent.run(query)
    print(result)

if __name__ == "__main__":
    main()

# Google News Agent py

In [ ]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import HumanMessage
from tools.spotify_tools import search_spotify_playlists
from config.setting import OPENAI_API_KEY

class GoogleNewsAgent:
    def __init__(self):
        self.openai_api_key = OPENAI_API_KEY
        self.model = ChatOpenAI(api_key=self.openai_api_key, model="gpt-4o-mini", verbose=True)
        self.agent = create_react_agent(
            model=self.model,
            tools=[search_news_by_keyword],
            prompt = (
              "You are a helpful Google search assistant. Your job is to help users find Google articles "
              "based on their search queries. Use the tool to perform a search and return the top result. "
              "Once the top result is found, return it as 'Final Answer: <result>' and stop."
            ),
            name="google_news_assistant"
        )

    def run(self, user_input):
        response = self.agent.invoke({
            "messages": [HumanMessage(content=user_input)]
        }, config={"recursion_limit": 50})
        return response["messages"][-1].content